<a href="https://colab.research.google.com/github/rotemefrati/deep-learning-assignment-11/blob/main/deep_learning_assignment_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Student Name: Rotem Efrati***

In [ ]:
import torch

**Question 1.a**

In [ ]:
def expand_as(A, B):
  """Mimics functionality of torch.Tensor.expand_as.

  Args:
    PyTorch tensors A, B.

  Returns:
    PyTorch tensor C: The broadcast of A to the size of B.

  Raises:
    ValueError: Tensor A is not broadcastable to tensor B.
  """
  # Check if A is broadcastable to B by broadcast rules.
  if not A.dim() <= B.dim():
    raise ValueError("Cannot broadcast A to B as the dimension of A is larger than the dimension of B")

  i, j = A.dim() - 1, B.dim() - 1
  while i >= 0:
    if A.size(i) != B.size(j) and A.size(i) != 1:
      raise ValueError("Tensor A is not broadcastable to B")  # Because of broadcast rules.
    i, j = i - 1, j - 1

  # If we got so far, A is broadcastable to B.
  C = A.clone()

  # If A has less dimensions than B, add dimensions to A.
  if C.dim() != B.dim():  # Then dim A is smaller than dim B.
    dims_to_add = B.dim() - C.dim()
    for _ in range(dims_to_add):
      C.unsqueeze_(0)  # We add sized-1 dimensions to the start of A, until it equals dim B.

  # Now A and B have the same amount of dimensions.
  i = C.dim() - 1
  while i >= 0:
    if C.size(i) != B.size(i):  # Then A's size at this dimension is 1.
      C = torch.cat([C] * B.size(i), i)  # Duplicate A along this dimension.
    i -= 1

  return C

**Question 1.b**

In [ ]:
def are_broadcastable_together(A, B):
  """Checks if tensors A and B are broadcastable together.

  Args:
    PyTorch tensors A, B.

  Returns:
    False, if tensors are not broadcastable.
    (True, torch.Size) containing the broadcasted shape, if tensors are broadcastable.
  """
  # Check if A and B are broadcastable by the general broadcast rules.
  k = min(A.dim(), B.dim()) - 1  # To iterate over the tensor with the smaller dimension.
  i, j = A.dim() - 1, B.dim() - 1
  while k >= 0:
    if A.size(i) != B.size(j) and A.size(i) != 1 and B.size(j) != 1:
      return False
    k, i, j = k-1, i-1, j-1

  # If we got so far, A and B are broadcastable.
  # Create a new list that will be containing the broadcasted shape.
  broadcasted_size = []

  k = min(A.dim(), B.dim()) - 1
  i, j = A.dim() - 1, B.dim() - 1
  # Take the larger dimension for the result.
  while k >= 0:
    if A.size(i) < B.size(j):
      broadcasted_size.insert(0, B.size(j))
    else:
      broadcasted_size.insert(0, A.size(i))
    k, i, j = k-1, i-1, j-1

  # Handle the remaining dimensions.
  for _ in range(abs(A.dim() - B.dim())):
    if A.dim() < B.dim():
      broadcasted_size.insert(0, B.size(j))
      j -= 1
    else:
      broadcasted_size.insert(0, A.size(i))
      i -= 1

  return True, torch.Size(broadcasted_size)

**Question 1.c**

In [ ]:
def broadcast_tensors(A, B):
  """Mimics functionality of torch.broadcast_tensors.

  Args:
    PyTorch tensors A, B.

  Returns:
    Two new tensors, which are the broadcast results of A and B.

  Raises:
    ValueError: If tensors are not broadcastable together.
  """
  # Check if tensors A, B are broadcastable together.
  result = are_broadcastable_together(A, B)

  if result is False:
    raise ValueError("The tensors are not broadcastable together")
  else:
    broadcastable, broadcasted_size = result  # Unpack the result.

  # Create a new empty tensor with the broadcasted size.
  broadcasted_tensor = torch.empty(broadcasted_size)
  # Broadcast A and B to their new common size and return them.
  return expand_as(A, broadcasted_tensor), expand_as(B, broadcasted_tensor)

**Question 1.d**

In [ ]:
A1 = torch.tensor([1])
B1 = torch.tensor([1, 2, 3, 4])
print("A1:", A1, "B1:", B1, sep='\n')
print("A1 is broadcastable to B1:")
print(expand_as(A1, B1), A1.expand_as(B1), sep='\n')

A1:
tensor([1])
B1:
tensor([1, 2, 3, 4])
A1 is broadcastable to B1:
tensor([1, 1, 1, 1])
tensor([1, 1, 1, 1])


In [ ]:
A2 = torch.arange(3)
B2 = torch.arange(9).reshape(3,3)
print("A2:", A2, "B2:", B2, sep='\n')
print("A2 is broadcastable to B2:")
print(expand_as(A2, B2), A2.expand_as(B2), sep='\n')

A2:
tensor([0, 1, 2])
B2:
tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]])
A2 is broadcastable to B2:
tensor([[0, 1, 2],
        [0, 1, 2],
        [0, 1, 2]])
tensor([[0, 1, 2],
        [0, 1, 2],
        [0, 1, 2]])


In [ ]:
A3 = torch.arange(3).reshape(3,1)
B3 = torch.arange(4).reshape(1,4)
print("A3:", A3, "B3:", B3, sep='\n')
print("A3 and B3 are broadcastable together:")
myA3, myB3 = broadcast_tensors(A3, B3)
pyA3, pyB3 = torch.broadcast_tensors(A3, B3)
print(myA3, pyA3, myB3, pyB3, sep='\n')

A3:
tensor([[0],
        [1],
        [2]])
B3:
tensor([[0, 1, 2, 3]])
A3 and B3 are broadcastable together:
tensor([[0, 0, 0, 0],
        [1, 1, 1, 1],
        [2, 2, 2, 2]])
tensor([[0, 0, 0, 0],
        [1, 1, 1, 1],
        [2, 2, 2, 2]])
tensor([[0, 1, 2, 3],
        [0, 1, 2, 3],
        [0, 1, 2, 3]])
tensor([[0, 1, 2, 3],
        [0, 1, 2, 3],
        [0, 1, 2, 3]])


In [ ]:
A4 = torch.arange(6).reshape(3,2)
B4 = torch.arange(5).reshape(1,5)
print("A4:", A4, "B4:", B4, sep='\n')
print("A4 and B4 are not broadcastable together:")
try:
  broadcast_tensors(A4, B4)
except Exception as e:
  print(f"My function error: {e}")
try:
  torch.broadcast_tensors(A4, B4)
except Exception as e:
  print(f"PyTorch function error: {e}")

A4:
tensor([[0, 1],
        [2, 3],
        [4, 5]])
B4:
tensor([[0, 1, 2, 3, 4]])
A4 and B4 are not broadcastable together:
My function error: The tensors are not broadcastable together
PyTorch function error: The size of tensor a (2) must match the size of tensor b (5) at non-singleton dimension 1


In [ ]:
A5 = torch.arange(24).reshape(2,3,4)
B5 = torch.tensor([0,1,2,3])
print("A5:", A5, "B5:", B5, sep='\n')
print("A5 is not broadcastable to B5, but B5 is broadcastable to A5:")
try:
  A5.expand_as(B5)
except Exception as e:
  print(f"PyTorch function error: {e}")
try:
  expand_as(A5, B5)
except Exception as e:
  print(f"My function error: {e}")
print(B5.expand_as(A5), expand_as(B5, A5), sep='\n')

A5:
tensor([[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11]],

        [[12, 13, 14, 15],
         [16, 17, 18, 19],
         [20, 21, 22, 23]]])
B5:
tensor([0, 1, 2, 3])
A5 is not broadcastable to B5, but B5 is broadcastable to A5:
PyTorch function error: expand(torch.LongTensor{[2, 3, 4]}, size=[4]): the number of sizes provided (1) must be greater or equal to the number of dimensions in the tensor (3)
My function error: Cannot broadcast A to B as the dimension of A is larger than the dimension of B
tensor([[[0, 1, 2, 3],
         [0, 1, 2, 3],
         [0, 1, 2, 3]],

        [[0, 1, 2, 3],
         [0, 1, 2, 3],
         [0, 1, 2, 3]]])
tensor([[[0, 1, 2, 3],
         [0, 1, 2, 3],
         [0, 1, 2, 3]],

        [[0, 1, 2, 3],
         [0, 1, 2, 3],
         [0, 1, 2, 3]]])
